In [ ]:

import cv2
import numpy as np
import pytesseract

In [3]:
!pip install -r requirements.txt

  Using cached python_docx-1.1.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached PyYAML-6.0.1-cp39-cp39-win_amd64.whl.metadata (2.1 kB)
   ---------------------------------------- 0.0/199.4 kB ? eta -:--:--
   -------------------------------- ------- 163.8/199.4 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 199.4/199.4 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   - -------------------------------------- 0.1/3.8 MB 4.3 MB/s eta 0:00:01
   --- ------------------------------------ 0.3/3.8 MB 4.1 MB/s eta 0:00:01
   ---- ----------------------------------- 0.4/3.8 MB 4.2 MB/s eta 0:00:01
   ----- ---------------------------------- 0.5/3.8 MB 3.3 MB/s eta 0:00:01
   ------- -------------------------------- 0.7/3.8 MB 3.1 MB/s eta 0:00:01
   -------- ------------------------------- 0.8/3.8 MB 3.4 MB/s eta 0:00:01
   ---------- ----------------------------- 1.0/3.8 MB 3.5 MB/s eta 0:00:01
   ------------- -------------

In [26]:
import fitz  
import cv2
import numpy as np
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' 

def convert_to_grayscale(image):
    if len(image.shape) == 2 or image.shape[2] == 1:
        return image
    elif image.shape[2] == 3:
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    elif image.shape[2] == 4:
        return cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)

def count_pixels_below_value(image, value):
    return np.sum(image < value)

def is_checkbox_marked(checkbox_region, threshold=0.5, dark_pixel_value=128):
    gray_region = convert_to_grayscale(checkbox_region)
    total_pixels = gray_region.size
    dark_pixels_count = count_pixels_below_value(gray_region, dark_pixel_value)
    dark_pixels_percentage = dark_pixels_count / total_pixels
    return dark_pixels_percentage > threshold

def process_image(image, page):
    gray = convert_to_grayscale(image)
    inverted = cv2.bitwise_not(gray)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.01 * cv2.arcLength(cnt, True), True)
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            checkbox_region = gray[y:y+h, x:x+w]
            marked = is_checkbox_marked(checkbox_region)
            text_x = x + w + 10  
            text_y = y + h / 2   
            if marked:
                page.insert_text((text_x, text_y), "en pré", fontsize=11, color=(0, 0, 1))
            else:
                page.insert_text((text_x, text_y), "en absence de", fontsize=11, color=(1, 0, 0))

pdf_path = 'test_0.pdf'
pdf_document = fitz.open(pdf_path)
for page_number in range(len(pdf_document)):
    page = pdf_document.load_page(page_number)
    zoom = 2
    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix=mat)
    image_np = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
    if pix.n == 4:
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGBA2BGR)
    elif pix.n == 1:
        image_np = image_np.reshape(pix.height, pix.width)
    else:
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
    process_image(image_np, page)
pdf_document.save("annotated_output.pdf")
pdf_document.close()
